# Задание по продуктовой аналитике

In [2]:
import pandas as pd

pd.set_option('display.max_columns', None)
deals_df = pd.read_pickle("deals_cleaned_v3.pkl")
deals_df

,Id,Deal Owner Name,Closing Date,Quality,Stage,Lost Reason,Page,Campaign,SLA,Content,Term,Source,Payment Type,Product,Education Type,Created Time,Course duration,Months of study,Initial Amount Paid,Offer Total Amount,Contact Name,City,Level of Deutsch,Created Date,Create_to_Close_Days,Latitude,Longitude,SLA_range
0,5805028000056864695,Ben Hall,NaT,B - Medium,New Lead,NaN,/eng/test,03.07.23women,NaT,v16,women,facebook ads,NaN,NaN,NaN,2024-06-21 15:30:00,NaN,<NA>,NaN,NaN,5805028000056849495,NaN,NaN,2024-06-21,NaN,NaN,NaN,NaN
1,5805028000056859489,Ulysses Adams,NaT,B - Medium,New Lead,NaN,/at-eng,NaN,NaT,NaN,NaN,organic,NaN,Web Developer,Morning,2024-06-21 15:23:00,6,<NA>,NaN,2000.0,5805028000056834471,NaN,NaN,2024-06-21,NaN,NaN,NaN,NaN
2,5805028000056832357,Ulysses Adams,2024-06-21,D - Non Target,Lost,Non target,/at-eng,engwien_at,0 days 00:26:43,b1-at,21_06_2024,telegram posts,NaN,NaN,NaN,2024-06-21 14:45:00,NaN,<NA>,NaN,NaN,5805028000056854421,NaN,NaN,2024-06-21,0.0,NaN,NaN,Medium (15-30 min)
3,5805028000056824246,Eva Kent,2024-06-21,E - Non Qualified,Lost,Invalid number,/eng,04.07.23recentlymoved_de,0 days 01:00:04,bloggersvideo14com,recentlymoved,facebook ads,NaN,NaN,NaN,2024-06-21 13:32:00,NaN,<NA>,NaN,NaN,5805028000056889351,NaN,NaN,2024-06-21,0.0,NaN,NaN,Very slow (>60 min)
4,5805028000056873292,Ben Hall,2024-06-21,D - Non Target,Lost,Non target,/eng,discovery_de,0 days 00:53:12,website,NaN,google ads,NaN,NaN,NaN,2024-06-21 13:21:00,NaN,<NA>,NaN,NaN,5805028000056876176,NaN,NaN,2024-06-21,0.0,NaN,NaN,Slow (30-60 min)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21406,5805028000050001603,Quincy Vincent,2024-05-15,E - Non Qualified,Lost,Duplicate,/eng/test,NaN,0 days 05:35:41,NaN,NaN,organic,NaN,NaN,NaN,2024-05-15 08:50:00,NaN,<NA>,NaN,NaN,5805028000049855125,NaN,NaN,2024-05-15,0.0,NaN,NaN,Very slow (>60 min)
21407,5805028000053787195,Paula Underwood,2024-06-06,E - Non Qualified,Lost,Duplicate,/eng/test,NaN,NaT,NaN,NaN,organic,NaN,NaN,NaN,2024-06-06 18:45:00,NaN,<NA>,NaN,NaN,5805028000053705642,NaN,NaN,2024-06-06,0.0,NaN,NaN,NaN
21408,5805028000054823001,Quincy Vincent,2024-06-12,E - Non Qualified,Lost,Duplicate,/eng,brand_search_eng_de,0 days 04:30:17,152789402780_{region_name}_668024583824,it hub,google ads,NaN,NaN,NaN,2024-06-12 05:46:00,NaN,<NA>,NaN,NaN,5805028000054480760,NaN,NaN,2024-06-12,0.0,NaN,NaN,Very slow (>60 min)
21409,5805028000056685164,Charlie Davis,2024-06-20,E - Non Qualified,Lost,Duplicate,/eng/test,NaN,0 days 00:01:46,NaN,NaN,organic,NaN,NaN,NaN,2024-06-20 13:49:00,NaN,<NA>,NaN,NaN,5805028000056392363,NaN,NaN,2024-06-20,0.0,NaN,NaN,Very fast (≤5 min)


In [3]:
deals_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21411 entries, 0 to 21410
Data columns (total 28 columns):
 #   Column                Non-Null Count  Dtype          
---  ------                --------------  -----          
 0   Id                    21411 non-null  object         
 1   Deal Owner Name       21382 non-null  object         
 2   Closing Date          16685 non-null  datetime64[ns] 
 3   Quality               21411 non-null  category       
 4   Stage                 21411 non-null  category       
 5   Lost Reason           15942 non-null  category       
 6   Page                  20342 non-null  object         
 7   Campaign              16006 non-null  category       
 8   SLA                   15504 non-null  timedelta64[ns]
 9   Content               14101 non-null  object         
 10  Term                  12387 non-null  object         
 11  Source                21411 non-null  object         
 12  Payment Type          496 non-null    category       
 13  P

$$
AOV_i = \frac{\text{Initial Amount Paid} + \left(\text{Offer Total Amount} - \text{Initial Amount Paid}\right)}{\text{Course duration} - 1} \cdot (\text{Months of study} - 1) \div \text{Months of study}
$$

In [4]:
deals_df['Course duration'] = pd.to_numeric(deals_df['Course duration'], errors='coerce')
deals_df['Course duration'] = deals_df['Course duration'].fillna(0).astype(int)

In [5]:
import numpy as np

# Условия для валидных строк
valid = (
    deals_df['Initial Amount Paid'].notna() &
    deals_df['Offer Total Amount'].notna() &
    deals_df['Course duration'].notna() &
    deals_df['Months of study'].notna() &
    (deals_df['Course duration'] != 1) &
    (deals_df['Months of study'] != 0)
)

# Расчёт AOV_i с приведением к float64
deals_df['AOV_i'] = np.where(
    valid,
    (
        deals_df['Initial Amount Paid'] +
        ((deals_df['Offer Total Amount'] - deals_df['Initial Amount Paid']) /
         (deals_df['Course duration'] - 1)) *
        (deals_df['Months of study'] - 1)
    ) / deals_df['Months of study'],
    np.nan
).astype('float64') #Т.к. тип данных получается Float64 (с большой буквы), то это может испортить данные (возвращает pd.NA вместо pd.nan и ломает вычисления даже при отсутствии строк с NaN), поэтому меняем тип на float64:

In [6]:
deals_df[deals_df['AOV_i'].notna()]

,Id,Deal Owner Name,Closing Date,Quality,Stage,Lost Reason,Page,Campaign,SLA,Content,Term,Source,Payment Type,Product,Education Type,Created Time,Course duration,Months of study,Initial Amount Paid,Offer Total Amount,Contact Name,City,Level of Deutsch,Created Date,Create_to_Close_Days,Latitude,Longitude,SLA_range,AOV_i
463,5805028000055502703,Eva Kent,2024-06-18,B - Medium,Payment Done,NaN,/pl-eng/web-developer,performancemax_wide_at,0 days 14:35:20,_{region_name}_,NaN,google ads,Recurring Payments,Digital Marketing,Evening,2024-06-15 20:01:00,11,1,500.0,4500.0,5805028000055471387,Podskalie,NaN,2024-06-15,3.0,48.1667,18.3167,Very slow (>60 min),500.000000
473,5805028000055524315,Ben Hall,2024-06-19,A - High,Payment Done,NaN,/eng,12.07.2023wide_de,0 days 00:17:25,bloggersvideo11,wide,tiktok ads,Recurring Payments,Digital Marketing,Evening,2024-06-15 17:02:00,11,1,500.0,4500.0,5805028000055503302,Zinnowitz,NaN,2024-06-15,4.0,54.0777,13.9104,Medium (15-30 min),500.000000
491,5805028000055524040,Ben Hall,2024-06-23,B - Medium,Payment Done,NaN,/eng,live_de,0 days 00:23:42,b0,03_06_2024,telegram posts,NaN,Web Developer,Morning,2024-06-15 12:20:00,6,1,1000.0,9000.0,5805028000055464156,Wolfsburg,B1,2024-06-15,8.0,52.4226,10.7865,Medium (15-30 min),1000.000000
824,5805028000054664018,Ben Hall,2024-06-19,D - Non Target,Payment Done,NaN,/eng,NaN,0 days 16:31:52,NaN,NaN,organic,NaN,Web Developer,Morning,2024-06-11 19:39:00,6,1,1000.0,9000.0,5805028000054683001,Zwickau,A2,2024-06-11,8.0,50.7194,12.4884,Very slow (>60 min),1000.000000
1029,5805028000054257317,Quincy Vincent,2024-06-17,B - Medium,Payment Done,NaN,/eng,02.07.23wide_de,0 days 00:45:55,bloggersvideo16com,wide,facebook ads,NaN,Digital Marketing,Morning,2024-06-10 15:33:00,11,1,1000.0,11000.0,5805028000054284461,Leinfelden-Echterdingen,B1,2024-06-10,7.0,48.6941,9.1479,Slow (30-60 min),1000.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21034,5805028000002302112,Julia Nelson,2023-07-19,C - Low,Payment Done,Expensive,eng/digital-marketing,performancemax_digitalmarkt_ru_de,0 days 15:24:52,_{region_name}_,NaN,google ads,NaN,Web Developer,Morning,2023-07-18 21:11:00,6,2,1000.0,5000.0,5805028000002302109,Wolfsburg,B1,2023-07-18,1.0,52.4226,10.7865,Very slow (>60 min),900.000000
21087,5805028000001987082,Julia Nelson,2023-07-17,C - Low,Payment Done,Conditions are not suitable,eng/digital-marketing,12.07.2023wide_de,0 days 00:49:16,v3com,wide,tiktok ads,NaN,Digital Marketing,Morning,2023-07-17 18:02:00,11,11,1000.0,11000.0,5805028000001986077,NaN,B1,2023-07-17,0.0,NaN,NaN,Slow (30-60 min),1000.000000
21130,5805028000001885076,Jane Smith,2023-08-31,A - High,Payment Done,NaN,eng/digital-marketing,04.07.23recentlymoved_de,0 days 00:13:31,b2,recentlymoved,facebook ads,One Payment,Digital Marketing,Morning,2023-07-15 13:27:00,11,11,450.0,4000.0,5805028000001880249,Ingolstadt,NaN,2023-07-15,47.0,48.7651,11.4237,Fast (5-15 min),363.636364
21265,5805028000001401001,Oliver Taylor,2023-07-16,B - Medium,Payment Done,NaN,eng/digital-marketing,02.07.23wide_de,0 days 02:22:36,b3,wide,facebook ads,NaN,Digital Marketing,NaN,2023-07-08 08:56:00,11,8,1000.0,11500.0,5805028000001350049,NaN,NaN,2023-07-08,8.0,NaN,NaN,Very slow (>60 min),1043.750000


$$
Revenue_i = AOV_i \times \text{Months of study}
$$

In [7]:
deals_df['Revenue_i'] = (deals_df['AOV_i'] * deals_df['Months of study']).astype('float64') #Т.к. тип данных получается Float64 (с большой буквы), то это может испортить данные (возвращает pd.NA вместо pd.nan и ломает вычисления даже при отсутствии строк с NaN), поэтому меняем тип на float64:

In [8]:
deals_df[deals_df['AOV_i'].notna()]

,Id,Deal Owner Name,Closing Date,Quality,Stage,Lost Reason,Page,Campaign,SLA,Content,Term,Source,Payment Type,Product,Education Type,Created Time,Course duration,Months of study,Initial Amount Paid,Offer Total Amount,Contact Name,City,Level of Deutsch,Created Date,Create_to_Close_Days,Latitude,Longitude,SLA_range,AOV_i,Revenue_i
463,5805028000055502703,Eva Kent,2024-06-18,B - Medium,Payment Done,NaN,/pl-eng/web-developer,performancemax_wide_at,0 days 14:35:20,_{region_name}_,NaN,google ads,Recurring Payments,Digital Marketing,Evening,2024-06-15 20:01:00,11,1,500.0,4500.0,5805028000055471387,Podskalie,NaN,2024-06-15,3.0,48.1667,18.3167,Very slow (>60 min),500.000000,500.0
473,5805028000055524315,Ben Hall,2024-06-19,A - High,Payment Done,NaN,/eng,12.07.2023wide_de,0 days 00:17:25,bloggersvideo11,wide,tiktok ads,Recurring Payments,Digital Marketing,Evening,2024-06-15 17:02:00,11,1,500.0,4500.0,5805028000055503302,Zinnowitz,NaN,2024-06-15,4.0,54.0777,13.9104,Medium (15-30 min),500.000000,500.0
491,5805028000055524040,Ben Hall,2024-06-23,B - Medium,Payment Done,NaN,/eng,live_de,0 days 00:23:42,b0,03_06_2024,telegram posts,NaN,Web Developer,Morning,2024-06-15 12:20:00,6,1,1000.0,9000.0,5805028000055464156,Wolfsburg,B1,2024-06-15,8.0,52.4226,10.7865,Medium (15-30 min),1000.000000,1000.0
824,5805028000054664018,Ben Hall,2024-06-19,D - Non Target,Payment Done,NaN,/eng,NaN,0 days 16:31:52,NaN,NaN,organic,NaN,Web Developer,Morning,2024-06-11 19:39:00,6,1,1000.0,9000.0,5805028000054683001,Zwickau,A2,2024-06-11,8.0,50.7194,12.4884,Very slow (>60 min),1000.000000,1000.0
1029,5805028000054257317,Quincy Vincent,2024-06-17,B - Medium,Payment Done,NaN,/eng,02.07.23wide_de,0 days 00:45:55,bloggersvideo16com,wide,facebook ads,NaN,Digital Marketing,Morning,2024-06-10 15:33:00,11,1,1000.0,11000.0,5805028000054284461,Leinfelden-Echterdingen,B1,2024-06-10,7.0,48.6941,9.1479,Slow (30-60 min),1000.000000,1000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21034,5805028000002302112,Julia Nelson,2023-07-19,C - Low,Payment Done,Expensive,eng/digital-marketing,performancemax_digitalmarkt_ru_de,0 days 15:24:52,_{region_name}_,NaN,google ads,NaN,Web Developer,Morning,2023-07-18 21:11:00,6,2,1000.0,5000.0,5805028000002302109,Wolfsburg,B1,2023-07-18,1.0,52.4226,10.7865,Very slow (>60 min),900.000000,1800.0
21087,5805028000001987082,Julia Nelson,2023-07-17,C - Low,Payment Done,Conditions are not suitable,eng/digital-marketing,12.07.2023wide_de,0 days 00:49:16,v3com,wide,tiktok ads,NaN,Digital Marketing,Morning,2023-07-17 18:02:00,11,11,1000.0,11000.0,5805028000001986077,NaN,B1,2023-07-17,0.0,NaN,NaN,Slow (30-60 min),1000.000000,11000.0
21130,5805028000001885076,Jane Smith,2023-08-31,A - High,Payment Done,NaN,eng/digital-marketing,04.07.23recentlymoved_de,0 days 00:13:31,b2,recentlymoved,facebook ads,One Payment,Digital Marketing,Morning,2023-07-15 13:27:00,11,11,450.0,4000.0,5805028000001880249,Ingolstadt,NaN,2023-07-15,47.0,48.7651,11.4237,Fast (5-15 min),363.636364,4000.0
21265,5805028000001401001,Oliver Taylor,2023-07-16,B - Medium,Payment Done,NaN,eng/digital-marketing,02.07.23wide_de,0 days 02:22:36,b3,wide,facebook ads,NaN,Digital Marketing,NaN,2023-07-08 08:56:00,11,8,1000.0,11500.0,5805028000001350049,NaN,NaN,2023-07-08,8.0,NaN,NaN,Very slow (>60 min),1043.750000,8350.0


In [9]:
print(deals_df['Revenue_i'].dtype)
print(deals_df['AOV_i'].dtype)

float64
float64


$$
AOV = \frac{\sum Revenue_i}{\sum \text{Months of study}}
$$

In [11]:
aov = (
    deals_df['Revenue_i'].sum(skipna=True) /
    deals_df['Months of study'].sum(skipna=True)
)
print(f"AOV = {aov:.2f}")

AOV = 783.21


In [12]:
contacts_df = pd.read_pickle("contacts_cleaned_v1.pkl")
contacts_df

,Id,Contact Owner Name,Created Time,Modified Time,Created Date,Created HourMinute
0,5805028000000645014,Rachel White,2023-06-27 11:28:00,2023-12-22 13:34:00,2023-06-27,11:28:00
1,5805028000000872003,Charlie Davis,2023-07-03 11:31:00,2024-05-21 10:23:00,2023-07-03,11:31:00
2,5805028000000889001,Bob Brown,2023-07-02 22:37:00,2023-12-21 13:17:00,2023-07-02,22:37:00
3,5805028000000907006,Bob Brown,2023-07-03 05:44:00,2023-12-29 15:20:00,2023-07-03,05:44:00
4,5805028000000939010,Nina Scott,2023-07-04 10:11:00,2024-04-16 16:14:00,2023-07-04,10:11:00
...,...,...,...,...,...,...
18543,5805028000056889209,Ulysses Adams,2024-06-21 12:11:00,2024-06-21 14:11:00,2024-06-21,12:11:00
18544,5805028000056889351,Eva Kent,2024-06-21 13:32:00,2024-06-21 15:32:00,2024-06-21,13:32:00
18545,5805028000056892018,Eva Kent,2024-06-21 10:21:00,2024-06-21 12:21:00,2024-06-21,10:21:00
18546,5805028000056892055,Yara Edwards,2024-06-21 10:22:00,2024-06-21 12:23:00,2024-06-21,10:22:00


In [13]:
spend_df = pd.read_pickle("spend_cleaned_v1.pkl")
spend_df

,Date,Source,Campaign,Impressions,Spend,Clicks,AdGroup,Ad
0,2023-07-03,google ads,gen_analyst_de,6,0.00,0,NaN,NaN
1,2023-07-03,google ads,performancemax_eng_de,4,0.01,1,NaN,NaN
2,2023-07-03,facebook ads,03.07.23women,187,3.30,6,women,b3
3,2023-07-03,facebook ads,03.07.23women,4,0.02,1,women,b1
4,2023-07-03,facebook ads,02.07.23wide_de,61,0.58,0,wide,b4
...,...,...,...,...,...,...,...,...
16395,2024-06-21,tiktok ads,22.05.2024wide_de,7,0.03,0,wide,bloggersvideo18com
16396,2024-06-21,youtube ads,youtube_shorts_de,90,0.51,2,com_august,bloggersvideo3june
16397,2024-06-21,facebook ads,17.03.24wide_at,7,0.07,0,wide,bloggersvideo16com_at
16398,2024-06-21,tiktok ads,12.07.2023wide_de,61,0.16,0,wide,bloggersvideo14com


In [14]:
deals_df['Contact Name'].nunique()

18089

## Расчет общей юнит-экономики

In [80]:
# -- поток юнитов масштабирования --
ua = contacts_df['Id'].nunique() 

# -- маректинговый бюджет -- 
ac = spend_df['Spend'].sum()

# -- стоимость привлечения юнита масштабирования --
ltc_cpa = ac / ua

# -- количество клиентов -- 
b = deals_df[
    (deals_df['Stage'] == 'Payment Done') &
    (deals_df['Initial Amount Paid'].notna())
]['Contact Name'].nunique()

# -- конверсия из лида в клиента -- 
c1 = b / ua

# -- количество транзакций --
t = deals_df['Months of study'].sum()

# -- среднее число сделок на одного клиента --
apc = t / b

# -- средняя валовая прибыль на клиента --
cltv = aov * apc

# -- средняя валовая прибыль на юнит масштабирования --
ltv = cltv * c1

# Стоимость привлечения клиента
cac = ac / b

# -- выручка --
revenue = b * aov * apc #deals_df['Revenue_i'].sum(skipna=True)

# -- маржинальная прибыль в юнит-экономике --
cm = ua * (ltv - ltc_cpa)

print(f'''AOV: {aov:.2f}
UA: {ua}
LTC (CPA): {ltc_cpa:.2f}
AC: {ac}
C1: {c1:.4f}
B: {b}
T: {t}
APC: {apc:.2f}
CLTV: {cltv:.2f}
LTV: {ltv:.2f}
CAC: {cac:.2f}
Revenue: {revenue:.2f}
CM: {cm:.2f}
''')

AOV: 783.21
UA: 18548
LTC (CPA): 8.06
AC: 149523.45
C1: 0.0446
B: 828
T: 4572
APC: 5.52
CLTV: 4324.66
LTV: 193.06
CAC: 180.58
Revenue: 3580815.00
CM: 3431291.55



In [81]:
import pandas as pd

def calculate_unit_economics_by_group(deals_df, contacts_df, spend_df):
    # Общие метрики
    ua_prod = contacts_df['Id'].nunique()
    ac_prod = spend_df['Spend'].sum()
    ltc_cpa_prod = ac_prod / ua_prod if ua_prod != 0 else 0

    results = []

    success_deals = deals_df[
    (deals_df['Stage'] == 'Payment Done') &
    (deals_df['Initial Amount Paid'].notna())
]

    # Группировка по Product и Education Type, без NaN
    grouped = success_deals.dropna(subset=['Product', 'Education Type']) \
                      .groupby(['Product', 'Education Type'], observed=True)

    for (product, edu_type), df_group in grouped:
        b_prod = df_group[
            (df_group['Stage'] == 'Payment Done') &
            (df_group['Initial Amount Paid'].notna())
        ]['Contact Name'].nunique()

        c1_prod = b_prod / ua_prod if ua_prod != 0 else 0
        t_prod = df_group['Months of study'].sum()
        apc_prod = t_prod / b_prod if b_prod != 0 else 0
        aov_prod = df_group['Revenue_i'].sum() / t_prod if t_prod != 0 else 0
        cltv_prod = aov_prod * apc_prod
        ltv_prod = cltv_prod * c1_prod
        cac_prod = ac_prod / b_prod
        revenue_prod = b_prod * aov_prod * apc_prod
        cm_prod = ua_prod * (ltv_prod - ltc_cpa_prod)
       
        results.append({
            'Product': product,
            'Education Type': edu_type,
            'AOV': aov_prod,
            'UA': ua_prod,
            'LTC (CPA)': ltc_cpa_prod,
            'AC': ac_prod,
            'C1': c1_prod,
            'B': b_prod,
            'T': t_prod,
            'APC': apc_prod,
            'CLTV': cltv_prod,
            'LTV': ltv_prod,   
            'CAC': cac_prod,
            'Revenue': revenue_prod,
            'CM': cm_prod
        })

    unit_economics_by_group_df = pd.DataFrame(results)
    return unit_economics_by_group_df

unit_df = calculate_unit_economics_by_group(deals_df, contacts_df, spend_df)
pd.set_option('display.float_format', '{:,.2f}'.format)
unit_df

,Product,Education Type,AOV,UA,LTC (CPA),AC,C1,B,T,APC,CLTV,LTV,CAC,Revenue,CM
0,Digital Marketing,Evening,365.69,18548,8.06,"149,523.45",0.01,112,764,6.82,"2,494.55",15.06,"1,335.03","279,390.00","129,866.55"
1,Digital Marketing,Morning,931.25,18548,8.06,"149,523.45",0.02,349,2056,5.89,"5,486.10",103.23,428.43,"1,914,650.00","1,765,126.55"
2,UX/UI Design,Evening,432.87,18548,8.06,"149,523.45",0.00,58,284,4.90,"2,119.57",6.63,"2,577.99","122,935.00","-26,588.45"
3,UX/UI Design,Morning,935.34,18548,8.06,"149,523.45",0.01,169,886,5.24,"4,903.61",44.68,884.75,"828,710.00","679,186.55"
4,Web Developer,Morning,726.10,18548,8.06,"149,523.45",0.01,135,505,3.74,"2,716.15",19.77,"1,107.58","366,680.00","217,156.55"


In [82]:
def change_metrics_table(df, product, education_type, delta_pct=10):
    increase_factor = 1 + delta_pct / 100
    decrease_factor = 1 - delta_pct / 100

    # Классификация метрик
    metrics = ['AOV', 'UA', 'AC', 'LTC (CPA)', 'C1', 'B', 'T', 'APC', 'CLTV', 'LTV', 'Revenue', 'CM']
    metrics_to_increase = ['AOV', 'UA', 'C1', 'APC']
    metrics_to_decrease = ['LTC (CPA)']
    metrics_fixed = ['AC', 'B', 'T', 'CLTV', 'LTV']  # рассчитываются, но не меняются напрямую
    row_metrics = metrics_to_increase + metrics_to_decrease

    # Получение базовых значений
    base = df[
        (df['Product'] == product) &
        (df['Education Type'] == education_type)
    ].iloc[0]
    base_values = base[metrics].to_dict()

    # Подготовка таблицы
    table = {
        'Метрика ↑ / ↓ Метрика →': ['Base'] + [
            f'{m} +{delta_pct}%' if m in metrics_to_increase else f'{m} -{delta_pct}%' for m in row_metrics
        ]
    }

    for col_metric in metrics:
        col_values = [base_values[col_metric]]

        for row_metric in row_metrics:
            temp = base_values.copy()

            # Изменение выбранной метрики
            if row_metric in metrics_to_increase:
                temp[row_metric] *= increase_factor
            elif row_metric in metrics_to_decrease:
                temp[row_metric] *= decrease_factor

            # Пересчёт зависимых метрик
            temp['CLTV'] = temp['AOV'] * temp['APC']
            temp['LTV'] = temp['CLTV'] * temp['C1']
            temp['CM'] = temp['UA'] * (temp['LTV'] - temp['LTC (CPA)'])
            temp['Revenue'] = temp['AOV'] * temp['APC'] * temp['B']

            col_values.append(temp[col_metric])

        table[col_metric] = col_values

    return pd.DataFrame(table)

In [83]:
change_metrics_table(unit_df, 'Digital Marketing', 'Evening', delta_pct=10)
# pd.set_option('display.float_format', '{:,.2f}'.format)

,Метрика ↑ / ↓ Метрика →,AOV,UA,AC,LTC (CPA),C1,B,T,APC,CLTV,LTV,Revenue,CM
0,Base,365.69,"18,548.00","149,523.45",8.06,0.01,112,764,6.82,"2,494.55",15.06,"279,390.00","129,866.55"
1,AOV +10%,402.26,"18,548.00","149,523.45",8.06,0.01,112,764,6.82,"2,744.01",16.57,"307,329.00","157,805.55"
2,UA +10%,365.69,"20,402.80","149,523.45",8.06,0.01,112,764,6.82,"2,494.55",15.06,"279,390.00","142,853.20"
3,C1 +10%,365.69,"18,548.00","149,523.45",8.06,0.01,112,764,6.82,"2,494.55",16.57,"279,390.00","157,805.55"
4,APC +10%,365.69,"18,548.00","149,523.45",8.06,0.01,112,764,7.50,"2,744.01",16.57,"307,329.00","157,805.55"
5,LTC (CPA) -10%,365.69,"18,548.00","149,523.45",7.26,0.01,112,764,6.82,"2,494.55",15.06,"279,390.00","144,818.89"


In [84]:
change_metrics_table(unit_df, 'Digital Marketing', 'Morning', delta_pct=10)

,Метрика ↑ / ↓ Метрика →,AOV,UA,AC,LTC (CPA),C1,B,T,APC,CLTV,LTV,Revenue,CM
0,Base,931.25,"18,548.00","149,523.45",8.06,0.02,349,2056,5.89,"5,486.10",103.23,"1,914,650.00","1,765,126.55"
1,AOV +10%,"1,024.38","18,548.00","149,523.45",8.06,0.02,349,2056,5.89,"6,034.71",113.55,"2,106,115.00","1,956,591.55"
2,UA +10%,931.25,"20,402.80","149,523.45",8.06,0.02,349,2056,5.89,"5,486.10",103.23,"1,914,650.00","1,941,639.21"
3,C1 +10%,931.25,"18,548.00","149,523.45",8.06,0.02,349,2056,5.89,"5,486.10",113.55,"1,914,650.00","1,956,591.55"
4,APC +10%,931.25,"18,548.00","149,523.45",8.06,0.02,349,2056,6.48,"6,034.71",113.55,"2,106,115.00","1,956,591.55"
5,LTC (CPA) -10%,931.25,"18,548.00","149,523.45",7.26,0.02,349,2056,5.89,"5,486.10",103.23,"1,914,650.00","1,780,078.89"


In [85]:
change_metrics_table(unit_df, 'UX/UI Design', 'Morning', delta_pct=10)

,Метрика ↑ / ↓ Метрика →,AOV,UA,AC,LTC (CPA),C1,B,T,APC,CLTV,LTV,Revenue,CM
0,Base,935.34,"18,548.00","149,523.45",8.06,0.01,169,886,5.24,"4,903.61",44.68,"828,710.00","679,186.55"
1,AOV +10%,"1,028.87","18,548.00","149,523.45",8.06,0.01,169,886,5.24,"5,393.97",49.15,"911,581.00","762,057.55"
2,UA +10%,935.34,"20,402.80","149,523.45",8.06,0.01,169,886,5.24,"4,903.61",44.68,"828,710.00","747,105.21"
3,C1 +10%,935.34,"18,548.00","149,523.45",8.06,0.01,169,886,5.24,"4,903.61",49.15,"828,710.00","762,057.55"
4,APC +10%,935.34,"18,548.00","149,523.45",8.06,0.01,169,886,5.77,"5,393.97",49.15,"911,581.00","762,057.55"
5,LTC (CPA) -10%,935.34,"18,548.00","149,523.45",7.26,0.01,169,886,5.24,"4,903.61",44.68,"828,710.00","694,138.90"


In [90]:
change_metrics_table(unit_df, 'UX/UI Design', 'Evening', delta_pct=10)

,Метрика ↑ / ↓ Метрика →,AOV,UA,AC,LTC (CPA),C1,B,T,APC,CLTV,LTV,Revenue,CM
0,Base,432.8697,"18,548.0000","149,523.4500",8.0614,0.0031,58,284,4.8966,"2,119.5690",6.6279,"122,935.0000","-26,588.4500"
1,AOV +10%,476.1567,"18,548.0000","149,523.4500",8.0614,0.0031,58,284,4.8966,"2,331.5259",7.2907,"135,228.5000","-14,294.9500"
2,UA +10%,432.8697,"20,402.8000","149,523.4500",8.0614,0.0031,58,284,4.8966,"2,119.5690",6.6279,"122,935.0000","-29,247.2950"
3,C1 +10%,432.8697,"18,548.0000","149,523.4500",8.0614,0.0034,58,284,4.8966,"2,119.5690",7.2907,"122,935.0000","-14,294.9500"
4,APC +10%,432.8697,"18,548.0000","149,523.4500",8.0614,0.0031,58,284,5.3862,"2,331.5259",7.2907,"135,228.5000","-14,294.9500"
5,LTC (CPA) -10%,432.8697,"18,548.0000","149,523.4500",7.2553,0.0031,58,284,4.8966,"2,119.5690",6.6279,"122,935.0000","-11,636.1050"


In [96]:
change_metrics_table(unit_df, 'UX/UI Design', 'Morning', delta_pct=10)

,Метрика ↑ / ↓ Метрика →,AOV,UA,AC,LTC (CPA),C1,B,T,APC,CLTV,LTV,Revenue,CM
0,Base,935.339,"18,548.000","149,523.450",8.061,0.009,169,886,5.243,"4,903.609",44.679,"828,710.000","679,186.550"
1,AOV +10%,"1,028.872","18,548.000","149,523.450",8.061,0.009,169,886,5.243,"5,393.970",49.147,"911,581.000","762,057.550"
2,UA +10%,935.339,"20,402.800","149,523.450",8.061,0.009,169,886,5.243,"4,903.609",44.679,"828,710.000","747,105.205"
3,C1 +10%,935.339,"18,548.000","149,523.450",8.061,0.010,169,886,5.243,"4,903.609",49.147,"828,710.000","762,057.550"
4,APC +10%,935.339,"18,548.000","149,523.450",8.061,0.009,169,886,5.767,"5,393.970",49.147,"911,581.000","762,057.550"
5,LTC (CPA) -10%,935.339,"18,548.000","149,523.450",7.255,0.009,169,886,5.243,"4,903.609",44.679,"828,710.000","694,138.895"


In [97]:
change_metrics_table(unit_df, 'Web Developer', 'Morning', delta_pct=10)	

,Метрика ↑ / ↓ Метрика →,AOV,UA,AC,LTC (CPA),C1,B,T,APC,CLTV,LTV,Revenue,CM
0,Base,726.099,"18,548.000","149,523.450",8.061,0.007,135,505,3.741,"2,716.148",19.769,"366,680.000","217,156.550"
1,AOV +10%,798.709,"18,548.000","149,523.450",8.061,0.007,135,505,3.741,"2,987.763",21.746,"403,348.000","253,824.550"
2,UA +10%,726.099,"20,402.800","149,523.450",8.061,0.007,135,505,3.741,"2,716.148",19.769,"366,680.000","238,872.205"
3,C1 +10%,726.099,"18,548.000","149,523.450",8.061,0.008,135,505,3.741,"2,716.148",21.746,"366,680.000","253,824.550"
4,APC +10%,726.099,"18,548.000","149,523.450",8.061,0.007,135,505,4.115,"2,987.763",21.746,"403,348.000","253,824.550"
5,LTC (CPA) -10%,726.099,"18,548.000","149,523.450",7.255,0.007,135,505,3.741,"2,716.148",19.769,"366,680.000","232,108.895"


In [95]:
pd.set_option('display.float_format', '{:,.3f}'.format)

In [105]:
import pandas as pd

def calculate_unit_economics_by_product(deals_df, contacts_df, spend_df):
    # Общие метрики
    total_ua = contacts_df['Id'].nunique()
    total_ac = spend_df['Spend'].sum()
    total_ltc_cpa = total_ac / total_ua if total_ua != 0 else 0

    results = []

    # Только успешные сделки
    success_deals = deals_df[
        (deals_df['Stage'] == 'Payment Done') &
        (deals_df['Initial Amount Paid'].notna())
    ]

    # Группировка только по Product, исключаем пустые значения
    grouped = success_deals.dropna(subset=['Product']).groupby('Product', observed=True)

    for product, group in grouped:
        buyers = group['Contact Name'].nunique()
        c1 = buyers / total_ua if total_ua != 0 else 0
        total_months = group['Months of study'].sum()
        apc = total_months / buyers if buyers != 0 else 0
        aov = group['Revenue_i'].sum() / total_months if total_months != 0 else 0
        cltv = aov * apc
        ltv = cltv * c1
        cac = total_ac / buyers if buyers != 0 else 0
        revenue = buyers * aov * apc
        cm = total_ua * (ltv - total_ltc_cpa)

        results.append({
            'Product': product,
            'AOV': aov,
            'UA': total_ua,
            'LTC (CPA)': total_ltc_cpa,
            'AC': total_ac,
            'C1': c1,
            'B': buyers,
            'T': total_months,
            'APC': apc,
            'CLTV': cltv,
            'LTV': ltv,
            'CAC': cac,
            'Revenue': revenue,
            'CM': cm
        })

    return pd.DataFrame(results)

product_df = calculate_unit_economics_by_product(deals_df, contacts_df, spend_df)
product_df

,Product,AOV,UA,LTC (CPA),AC,C1,B,T,APC,CLTV,LTV,CAC,Revenue,CM
0,Digital Marketing,783.954,18548,8.061,"149,523.450",0.025,468,2886,6.167,"4,834.380",121.980,319.495,"2,262,490.000","2,112,966.550"
1,Find yourself in IT,0.000,18548,8.061,"149,523.450",0.000,1,0,0.000,0.000,0.000,"149,523.450",0.000,"-149,523.450"
2,UX/UI Design,813.372,18548,8.061,"149,523.450",0.012,226,1170,5.177,"4,210.819",51.307,661.608,"951,645.000","802,121.550"
3,Web Developer,726.099,18548,8.061,"149,523.450",0.007,135,505,3.741,"2,716.148",19.769,"1,107.581","366,680.000","217,156.550"


In [106]:
def simulate_metric_changes_by_product(df, product_name, delta_pct=10):
    increase_factor = 1 + delta_pct / 100
    decrease_factor = 1 - delta_pct / 100

    # Классификация метрик
    metrics = ['AOV', 'UA', 'AC', 'LTC (CPA)', 'C1', 'B', 'T', 'APC', 'CLTV', 'LTV', 'Revenue', 'CM']
    metrics_to_increase = ['AOV', 'UA', 'C1', 'APC']
    metrics_to_decrease = ['LTC (CPA)']
    metrics_fixed = ['AC', 'B', 'T', 'CLTV', 'LTV']  # рассчитываются, но не меняются напрямую
    row_metrics = metrics_to_increase + metrics_to_decrease

    # Получение базовых значений
    base_row = df[df['Product'] == product_name].iloc[0]
    base_values = base_row[metrics].to_dict()

    # Подготовка таблицы
    table = {
        'Metric ↑ / ↓ →': ['Base'] + [
            f'{m} +{delta_pct}%' if m in metrics_to_increase else f'{m} -{delta_pct}%' for m in row_metrics
        ]
    }

    for column_metric in metrics:
        values = [base_values[column_metric]]

        for changed_metric in row_metrics:
            temp = base_values.copy()

            # Применение изменений
            if changed_metric in metrics_to_increase:
                temp[changed_metric] *= increase_factor
            elif changed_metric in metrics_to_decrease:
                temp[changed_metric] *= decrease_factor

            # Пересчёт зависимых метрик
            temp['CLTV'] = temp['AOV'] * temp['APC']
            temp['LTV'] = temp['CLTV'] * temp['C1']
            temp['CM'] = temp['UA'] * (temp['LTV'] - temp['LTC (CPA)'])
            temp['Revenue'] = temp['AOV'] * temp['APC'] * temp['B']

            values.append(temp[column_metric])

        table[column_metric] = values

    return pd.DataFrame(table)

In [107]:
simulate_metric_changes_by_product(product_df, 'Digital Marketing', delta_pct=10)

,Metric ↑ / ↓ →,AOV,UA,AC,LTC (CPA),C1,B,T,APC,CLTV,LTV,Revenue,CM
0,Base,783.954,"18,548.000","149,523.450",8.061,0.025,468,2886,6.167,"4,834.380",121.980,"2,262,490.000","2,112,966.550"
1,AOV +10%,862.349,"18,548.000","149,523.450",8.061,0.025,468,2886,6.167,"5,317.818",134.178,"2,488,739.000","2,339,215.550"
2,UA +10%,783.954,"20,402.800","149,523.450",8.061,0.025,468,2886,6.167,"4,834.380",121.980,"2,262,490.000","2,324,263.205"
3,C1 +10%,783.954,"18,548.000","149,523.450",8.061,0.028,468,2886,6.167,"4,834.380",134.178,"2,262,490.000","2,339,215.550"
4,APC +10%,783.954,"18,548.000","149,523.450",8.061,0.025,468,2886,6.783,"5,317.818",134.178,"2,488,739.000","2,339,215.550"
5,LTC (CPA) -10%,783.954,"18,548.000","149,523.450",7.255,0.025,468,2886,6.167,"4,834.380",121.980,"2,262,490.000","2,127,918.895"


In [108]:
simulate_metric_changes_by_product(product_df, 'UX/UI Design', delta_pct=10)

,Metric ↑ / ↓ →,AOV,UA,AC,LTC (CPA),C1,B,T,APC,CLTV,LTV,Revenue,CM
0,Base,813.372,"18,548.000","149,523.450",8.061,0.012,226,1170,5.177,"4,210.819",51.307,"951,645.000","802,121.550"
1,AOV +10%,894.709,"18,548.000","149,523.450",8.061,0.012,226,1170,5.177,"4,631.900",56.438,"1,046,809.500","897,286.050"
2,UA +10%,813.372,"20,402.800","149,523.450",8.061,0.012,226,1170,5.177,"4,210.819",51.307,"951,645.000","882,333.705"
3,C1 +10%,813.372,"18,548.000","149,523.450",8.061,0.013,226,1170,5.177,"4,210.819",56.438,"951,645.000","897,286.050"
4,APC +10%,813.372,"18,548.000","149,523.450",8.061,0.012,226,1170,5.695,"4,631.900",56.438,"1,046,809.500","897,286.050"
5,LTC (CPA) -10%,813.372,"18,548.000","149,523.450",7.255,0.012,226,1170,5.177,"4,210.819",51.307,"951,645.000","817,073.895"


In [109]:
simulate_metric_changes_by_product(product_df, 'Web Developer', delta_pct=10)

,Metric ↑ / ↓ →,AOV,UA,AC,LTC (CPA),C1,B,T,APC,CLTV,LTV,Revenue,CM
0,Base,726.099,"18,548.000","149,523.450",8.061,0.007,135,505,3.741,"2,716.148",19.769,"366,680.000","217,156.550"
1,AOV +10%,798.709,"18,548.000","149,523.450",8.061,0.007,135,505,3.741,"2,987.763",21.746,"403,348.000","253,824.550"
2,UA +10%,726.099,"20,402.800","149,523.450",8.061,0.007,135,505,3.741,"2,716.148",19.769,"366,680.000","238,872.205"
3,C1 +10%,726.099,"18,548.000","149,523.450",8.061,0.008,135,505,3.741,"2,716.148",21.746,"366,680.000","253,824.550"
4,APC +10%,726.099,"18,548.000","149,523.450",8.061,0.007,135,505,4.115,"2,987.763",21.746,"403,348.000","253,824.550"
5,LTC (CPA) -10%,726.099,"18,548.000","149,523.450",7.255,0.007,135,505,3.741,"2,716.148",19.769,"366,680.000","232,108.895"


In [110]:
simulate_metric_changes_by_product(product_df, 'Find yourself in IT', delta_pct=10)

,Metric ↑ / ↓ →,AOV,UA,AC,LTC (CPA),C1,B,T,APC,CLTV,LTV,Revenue,CM
0,Base,0.000,"18,548.000","149,523.450",8.061,0.000,1,0,0.000,0.000,0.000,0.000,"-149,523.450"
1,AOV +10%,0.000,"18,548.000","149,523.450",8.061,0.000,1,0,0.000,0.000,0.000,0.000,"-149,523.450"
2,UA +10%,0.000,"20,402.800","149,523.450",8.061,0.000,1,0,0.000,0.000,0.000,0.000,"-164,475.795"
3,C1 +10%,0.000,"18,548.000","149,523.450",8.061,0.000,1,0,0.000,0.000,0.000,0.000,"-149,523.450"
4,APC +10%,0.000,"18,548.000","149,523.450",8.061,0.000,1,0,0.000,0.000,0.000,0.000,"-149,523.450"
5,LTC (CPA) -10%,0.000,"18,548.000","149,523.450",7.255,0.000,1,0,0.000,0.000,0.000,0.000,"-134,571.105"
